```
    ██╗   ██╗ ██████╗ ██╗      █████╗  ██████╗████████╗
    ╚██╗ ██╔╝██╔═══██╗██║     ██╔══██╗██╔════╝╚══██╔══╝
     ╚████╔╝ ██║   ██║██║     ███████║██║        ██║   
      ╚██╔╝  ██║   ██║██║     ██╔══██║██║        ██║   
       ██║   ╚██████╔╝███████╗██║  ██║╚██████╗   ██║   
       ╚═╝    ╚═════╝ ╚══════╝╚═╝  ╚═╝ ╚═════╝   ╚═╝ 
```


In [18]:
from utils import VideoReaders, DetectorLoader
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from skimage import segmentation
from itertools import groupby
import random, warnings
from SORT.sort import *
from skimage import measure, color, segmentation, draw
import math
from scipy.spatial.distance import cdist

# Step 1: Identify True Positives

In [19]:
ground_truths = pd.read_csv('datasets/slip-fall/mladen-holdout/slip-fall-mladen-ground-truth.csv')
ground_truths[ground_truths['Fall?'] == 1]

,File,Fall?
0,0002.mp4,1
10,0012.mp4,1
11,0013.mp4,1
12,0014.mp4,1
18,0020.mp4,1
19,0021.mp4,1
20,0022.mp4,1
21,0023.mp4,1
22,0024.mp4,1
23,0025.mp4,1


# Step 2: Load input video


In [23]:
# Instantiate model
path = '../datasets/slip-fall/mladen-holdout/05-FPS/0.25x/0021.mp4'
frame_provider = VideoReaders.VideoReader(path)
length, shape = frame_provider.properties()
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    model = DetectorLoader.YOLACT('./weights/slip-fall-chosen-weights/yolact_resnet50_432_74900.pth', threshold = 0.25)

In [24]:
frames = []
inference = []

for frame in frame_provider:
    frames.append(frame[:,:,[2,1,0]])
    c, s, bb, ma = model.predict(frame[:,:,[2,1,0]])
    idx = np.where(c == 0) # Person has a Class ID of 0
    pixelwise_arrays = []
    for item in idx:
        for n, qq in enumerate(item):
            pixelwise = np.zeros_like(ma[qq,...])
            pixelwise = ma[qq,...].astype(np.uint8)
            pixelwise[np.where(pixelwise == 1)] = n + 1
            pixelwise_arrays.append(pixelwise)

    filtered = []
    for array in pixelwise_arrays:
        if np.max(array) != 0:
            filtered.append(array)

    stacked = np.sum(filtered, axis = 0) # Will cause the array pixel values to increase in numerical value. This also maintains multiple people in the frame. 
    inference.append(stacked)

output = []
for arry in inference: # Check if the shape coming out of the model is OK.
    if len(arry.shape) < 2:
        output.append(np.zeros(shape).astype(np.uint8))
    else:
        output.append(arry)

def check(list1, val):
    return(any(x < val for x in list1))

# Remove anything that's not in the frame
for frame_position in range(0, length):
    generated_image = np.zeros_like(output[frame_position])
    for pixel in np.unique(output[frame_position]):
        if pixel == 0: 
            pass
        else:
            ignore_perimeter = 25 # Tunable parameter for ignore zone
            end_extent = (generated_image.shape[0] - ignore_perimeter, generated_image.shape[1] - ignore_perimeter)
            perim_rr, perim_cc = draw.rectangle_perimeter(start = (ignore_perimeter, ignore_perimeter), end = end_extent)
            rectangle_coords = np.stack((perim_rr, perim_cc), axis = 1)
            generated_image[rectangle_coords[:,0], rectangle_coords[:,1]] = 1 # Debugging
            for otherregion in measure.regionprops((output[frame_position] == pixel).astype(np.uint8)):
                mask_coordinates = otherregion.coords
                generated_image[mask_coordinates[:,0], mask_coordinates[:,1]] = 1
            distances = cdist(mask_coordinates, rectangle_coords, metric = 'euclidean') # Compute the euclidean distance between points and imposed boundary 
            minimum_distance = np.min(distances, axis = 1).tolist()
            integer_minimum_distances = [int(item) for item in minimum_distance]
            if check(integer_minimum_distances, 1):
                output[frame_position][output[frame_position] == pixel] = 0

In [25]:
# Create segmentation boundary mask for printout
if len(output) != len(frames):
    raise ValueError("Frame Lengths are off.")
boundaries = []
for im, mask in zip(frames, output):
    boundaries.append(segmentation.mark_boundaries(im, mask, mode = 'thick', color = (0,1,0)))

# Create matplotlib. 
columns = 8
rows, r = divmod(length, columns)
if r:
    rows += 1
f, ax = plt.subplots(rows, columns, figsize = (50,60))
ax = ax.flatten()
for n in range(0, length):
    ax[n].imshow(boundaries[n], aspect = 'auto')
    current_frame = str(n)
    ax[n].set(title = 'Frame {}'.format(current_frame.zfill(4)))
    ax[n].axis('off')
f.tight_layout()
f.savefig('{}.png'.format(os.path.basename(path).replace('.mp4', '')), dpi = 100)
plt.close(f)

In [26]:
# Derive measurement of angles
minimal_dictionary = []
for i in range(0, length):
    for region in measure.regionprops(output[i], color.rgb2gray(frames[i])):
        minimal_dictionary.append({
            "Frame": i, 
            "ID": region.label, 
            "BBOX": region.bbox
        })
tracked_bboxes = []
Sorter = Sort(max_age = 5, min_hits = 1, iou_threshold = 0.1) # 1 Tunable parameter (iou_threshold)
for instance, entry in groupby(minimal_dictionary, key = lambda x:x['Frame']):
    entry_list = []
    for item in entry:
        lst = list(item['BBOX'])
        lst.extend([0, item['ID']])
        entry_list.append(lst)
    track_bbs_ids = Sorter.update(np.array(entry_list))
    for objects in track_bbs_ids:
        r0, c0, r1, c1, ID, label = objects.tolist()
        for region in measure.regionprops(output[instance], color.rgb2gray(frames[instance])):
            if region.label == label:
                this_normalized_moment = region.moments_normalized
                angle = np.degrees((np.arctan2(2*this_normalized_moment[1,1], this_normalized_moment[2,0] - this_normalized_moment[0,2]))/2)
                w = region.bbox[2] - region.bbox[0]
                h = region.bbox[3] - region.bbox[1]
                ar = w / float(h)
                tracked_bboxes.append({'Frame': instance, 
                                        'ID': int(label), 
                                        'Track ID': int(ID), 
                                        'BBOX': region.bbox,
                                        'Angle': angle,
                                        'Area': region.area,
                                        'Aspect Ratio': ar, 
                                        'Eccentricity': region.eccentricity, 
                                        'Perimeter': region.perimeter})
tracked_dataframe = pd.DataFrame(tracked_bboxes)

In [29]:
class ChainedAssignment:
    def __init__(self, chained=None):
        acceptable = [None, 'warn', 'raise']
        assert chained in acceptable, "chained must be in " + str(acceptable)
        self.swcw = chained

    def __enter__(self):
        self.saved_swcw = pd.options.mode.chained_assignment
        pd.options.mode.chained_assignment = self.swcw
        return self

    def __exit__(self, *args):
        pd.options.mode.chained_assignment = self.saved_swcw

In [35]:
super_output = []
if len(tracked_dataframe) == 0:
    super_output.append({'File': None, 'Frame': None, 'Bounding Box': None})
else:
    for tracks, track_df in tracked_dataframe.groupby('Track ID'):
        current_track = track_df.loc[(track_df['Track ID'] == tracks), :]
        with ChainedAssignment():
            current_track['Δ Angle'] = current_track.loc[(current_track['Track ID'] == tracks), :]['Angle'].diff(periods = 1) # 1 Tunable parameters (Periodicity for rate of change in angle)
            current_track['Δ Perimeter'] = current_track.loc[(current_track['Track ID'] == tracks), :]['Perimeter'].diff()
            # current_track['Slip-Fall'] = np.where(current_track['Δ Change'] < -3, 'Slip', 'Stand') # 1 Tunable Parameter (Hard Cutoff for % change in angle)
            display(current_track)
        if len(current_track.loc[current_track['Slip-Fall'] == 'Slip']) != 0:
            super_output.append({'Track ID': tracks, 'File': os.path.basename(path), 'Frame': current_track.loc[current_track['Slip-Fall'] == 'Slip'].iloc[0]['Frame'], 'Bounding Box': current_track.loc[current_track['Slip-Fall'] == 'Slip'].iloc[0]['BBOX']})
        else:
            super_output.append({'Track ID': None, 'File': None, 'Frame': None, 'Bounding Box': None})

fall_ids = pd.DataFrame(super_output)

,Frame,ID,Track ID,BBOX,Angle,Area,Aspect Ratio,Eccentricity,Perimeter,Δ Angle,Δ Perimeter
0,10,1,1,"(73, 480, 211, 700)",89.867270,18785,0.627273,0.800817,764.014285,NaN,NaN
1,11,1,1,"(60, 482, 297, 675)",26.833706,21392,1.227979,0.716595,770.055916,-63.033565,6.041631
2,12,1,1,"(60, 494, 368, 638)",5.190507,22320,2.138889,0.903381,956.624458,-21.643199,186.568542
3,13,1,1,"(82, 492, 429, 625)",-0.684249,17764,2.609023,0.940622,1148.560533,-5.874756,191.936075
4,14,1,1,"(111, 487, 438, 630)",-0.840584,20581,2.286713,0.926387,925.234631,-0.156334,-223.325902
5,15,1,1,"(129, 486, 432, 640)",-5.386614,21697,1.967532,0.903629,874.749350,-4.546030,-50.485281
6,16,1,1,"(136, 489, 435, 648)",-5.298552,22632,1.880503,0.870412,834.950360,0.088062,-39.798990
7,17,1,1,"(139, 492, 438, 654)",-0.135998,21831,1.845679,0.863480,835.335137,5.162554,0.384776
8,18,1,1,"(139, 513, 469, 654)",3.180263,21562,2.340426,0.905821,895.394011,3.316261,60.058875
9,19,1,1,"(140, 515, 483, 655)",2.869463,21945,2.450000,0.924172,940.666089,-0.310800,45.272078


KeyError: 'Slip-Fall'